In [2]:

# from sklearn import datasets
# iris = datasets.load_iris()
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn import metrics, tree, svm
from sklearn.feature_selection import chi2, SelectKBest
import arff
import itertools
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

In [3]:
import numpy as np
import pandas as pd
import os, re
from ast import literal_eval
os.getcwd()

'/home/tchanchokpong/Dropbox/MIT/KnowledgeTechnology/Proj2'

In [4]:
###### Extracting data ###### For feature engineering
trainTwt = []
with open("./2017S1-KTproj2-data/train-tweets.txt") as f:
    for line in f:
        trainTwt.append(line)

In [5]:
trainLabel = []
with open("./2017S1-KTproj2-data/train-labels.txt") as f:
    for line in f:
        trainLabel.append(line)

In [6]:
twtId = []
twtTxt = []
twtLabel = []
for i in range(len(trainTwt)):
    twtId.append(trainTwt[i].split("\t")[0])
    twtTxt.append(trainTwt[i].split("\t")[1].rstrip())
    twtLabel.append(trainLabel[i].split("\t")[1][0:-1])

In [7]:
# this cannot replicate the data provided by the course (might be trivial differences in string format)
# sum(1 for _ in re.finditer(r'\b%s\b' % re.escape("a"), "Mary has a lamb"))

In [8]:
# using pandas, read directly from csv which is converted from ARFF

In [9]:
twtDf = pd.read_csv("/home/tchanchokpong/Dropbox/MIT/KnowledgeTechnology/Proj2/2017S1-KTproj2-data/train.csv")

In [10]:
sentiment = twtDf['sentiment']
twtDf.drop('sentiment', axis = 1, inplace = True)
twtDf.drop('id', axis = 1, inplace = True)
twtDf.head()

,a,amazing,antman,are,at,awesome,best,birthday,cant,cream,...,racist,see,shit,so,stupid,supremacists,th,they,tomorrow,trump
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [11]:
sentiment.value_counts()

neutral     11454
positive     6471
negative     5062
Name: sentiment, dtype: int64

In [12]:
devel = pd.read_csv("./2017S1-KTproj2-data/dev.csv")

In [13]:
clf = GaussianNB() # assume normal distribution
clf.fit(twtDf,sentiment)

GaussianNB(priors=None)

In [14]:
devForPred = devel[devel.columns[0:-1]]
devSent = devel[devel.columns[-1]]

In [15]:
predNaiveDevel = clf.predict(devForPred)

In [16]:
NaiveCM = confusion_matrix(devSent, predNaiveDevel, labels = ["positive","negative","neutral"])
NaiveCM

array([[ 874,   48,  566],
       [ 188,  312,  538],
       [ 629,  226, 1545]])

In [17]:
pd.Series(predNaiveDevel).value_counts()

neutral     2649
positive    1691
negative     586
dtype: int64

In [18]:
print(classification_report(devSent,predNaiveDevel, labels = ["positive","negative","neutral"]))

             precision    recall  f1-score   support

   positive       0.52      0.59      0.55      1488
   negative       0.53      0.30      0.38      1038
    neutral       0.58      0.64      0.61      2400

avg / total       0.55      0.55      0.55      4926



In [19]:
accuracy_score(devSent,predNaiveDevel)

0.5544051969143321

In [20]:
# Multinomial Bayes
clfMulti = MultinomialNB()
clfMulti.fit(twtDf, sentiment)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
predMultiNB = clfMulti.predict(devForPred)

In [22]:
MultiNBCM = confusion_matrix(devSent,predMultiNB, labels = ["positive","negative","neutral"])
MultiNBCM

array([[ 510,   26,  952],
       [  55,  225,  758],
       [ 154,  115, 2131]])

In [23]:
pd.Series(predMultiNB).value_counts()

neutral     3841
positive     719
negative     366
dtype: int64

In [24]:
pd.Series(devSent).value_counts()

neutral     2400
positive    1488
negative    1038
Name: sentiment, dtype: int64

In [25]:
print(classification_report(devSent,predMultiNB, labels = ["positive","negative","neutral"]))

             precision    recall  f1-score   support

   positive       0.71      0.34      0.46      1488
   negative       0.61      0.22      0.32      1038
    neutral       0.55      0.89      0.68      2400

avg / total       0.61      0.58      0.54      4926



In [26]:
accuracy_score(devSent,predMultiNB)
# we see marked improvedment (why?)

0.58181079983759643

In [27]:
0.58181079983759643 - 0.5544051969143321 # 2.74% improvements

0.027405602923264327

In [28]:
### Decision trees 

In [29]:
treeClf = tree.DecisionTreeClassifier()
treeClf.fit(twtDf, sentiment) # this could overfit the data 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [30]:
# check for overfitting by predicting on itself and see how it does with development
predTrainTree = treeClf.predict(twtDf) 

In [31]:
print(classification_report(sentiment,predTrainTree,labels = ["positive","negative","neutral"]))

             precision    recall  f1-score   support

   positive       0.84      0.43      0.57      6471
   negative       0.73      0.39      0.51      5062
    neutral       0.62      0.92      0.74     11454

avg / total       0.71      0.67      0.64     22987



In [32]:
predDevTree = treeClf.predict(devForPred)

In [33]:
confusion_matrix(devSent, predDevTree,labels = ["positive","negative","neutral"])

array([[ 479,   92,  917],
       [  65,  246,  727],
       [ 183,  202, 2015]])

In [34]:
print(classification_report(devSent,predDevTree,labels = ["positive","negative","neutral"]))

             precision    recall  f1-score   support

   positive       0.66      0.32      0.43      1488
   negative       0.46      0.24      0.31      1038
    neutral       0.55      0.84      0.67      2400

avg / total       0.56      0.56      0.52      4926



In [35]:
## possible overfitting problems / 
accuracy_score(devSent,predDevTree)

0.55623223710921643

In [36]:
# Retrain with maximum depth
treeClfM = tree.DecisionTreeClassifier(max_depth = 32) # from 46 features, set maximum at around 30
treeClfM.fit(twtDf,sentiment)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=32,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [37]:
predDevTreeM = treeClfM.predict(devForPred)

In [38]:
confusion_matrix(devSent, predDevTreeM,labels = ["positive","negative","neutral"])

array([[ 448,   58,  982],
       [  47,  191,  800],
       [ 133,  124, 2143]])

In [39]:
print(classification_report(devSent,predDevTreeM,labels = ["positive","negative","neutral"]))

             precision    recall  f1-score   support

   positive       0.71      0.30      0.42      1488
   negative       0.51      0.18      0.27      1038
    neutral       0.55      0.89      0.68      2400

avg / total       0.59      0.56      0.52      4926



In [40]:
accuracy_score(devSent, predDevTreeM) # higher, avoiding overfit, outperformed by NaiveBayes Multinomial

0.5647584246853431

In [41]:
RFclf = RandomForestClassifier(max_depth = 32)
RFclf.fit(twtDf, sentiment)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=32, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [42]:
predRF = RFclf.predict(devForPred)

In [43]:
print(classification_report(devSent,predRF, labels = ["positive","negative","neutral"])) # better recall 

             precision    recall  f1-score   support

   positive       0.67      0.36      0.47      1488
   negative       0.55      0.19      0.28      1038
    neutral       0.56      0.87      0.68      2400

avg / total       0.59      0.57      0.53      4926



In [44]:
confusion_matrix(devSent, predRF,labels = ["positive","negative","neutral"])

array([[ 533,   49,  906],
       [  74,  195,  769],
       [ 192,  110, 2098]])

In [45]:
accuracy_score(devSent,predRF) # slight improvement from lower biases

0.57369062119366621

In [46]:
# Export Decision Trees

In [47]:
with open("tree.dot", 'w') as f:
    f = tree.export_graphviz(treeClfM, out_file=f)

In [48]:
############ SVM ###############

In [49]:
svmClf = svm.LinearSVC()
svmClf.fit(twtDf,sentiment)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [50]:
predLSVC = svmClf.predict(devForPred)

In [51]:
confusion_matrix(devSent, predLSVC,labels = ["positive","negative","neutral"])

array([[ 542,   25,  921],
       [  60,  214,  764],
       [ 155,   94, 2151]])

In [52]:
print(classification_report(devSent,predLSVC,labels = ["positive","negative","neutral"]))

             precision    recall  f1-score   support

   positive       0.72      0.36      0.48      1488
   negative       0.64      0.21      0.31      1038
    neutral       0.56      0.90      0.69      2400

avg / total       0.62      0.59      0.55      4926



In [53]:
confusion_matrix(devSent,predLSVC, labels = ["positive","negative","neutral"])

array([[ 542,   25,  921],
       [  60,  214,  764],
       [ 155,   94, 2151]])

In [54]:
accuracy_score(devSent,predLSVC)

0.59013398294762487

In [55]:
# svmClfRBF = svm.SVC()
# svmClfRBF.fit(twtDf,sentiment)

In [56]:
# predSVCRBF = svmClfRBF.predict(devForPred)

In [57]:
# print(classification_report(devSent,predLSVC,labels = ["positive","negative","neutral"]))

In [58]:
# accuracy_score(devSent,predSVCRBF) # better off with linear (perhaps)

In [59]:
######################################### Some sanity Checks ##################################################

In [60]:
twtTxt[2223]

'".@lamiebony If this\'s true, resignation will be in order. They may eliminate her in Borno and say it\'s Boko Haram https://t.co/hF2QK1NNus"'

In [61]:
literal_eval(twtTxt[2223]) # there aren't many and should not really affect the tagging

".@lamiebony If this's true, resignation will be in order. They may eliminate her in Borno and say it's Boko Haram https://t.co/hF2QK1NNus"

In [62]:
twtTxtliteral = []
error = []
for i in range(len(twtTxt)):
    try:
        twtTxtliteral.append(literal_eval(twtTxt[i]))
    except (EOFError,SyntaxError,ValueError) :
        error.append(i)
#        print(i)
        twtTxtliteral.append(twtTxt[i])

In [63]:
len(twtTxtliteral)

22987

In [64]:
# features

In [65]:
######################################### Features Engineering ##################################################

In [66]:
# Part of speech tagging (POS tagging) using tree tagger
import treetaggerwrapper

In [67]:
# test
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')

In [68]:
treetaggerwrapper.make_tags(tagger.tag_text(twtTxt[0]))

[Tag(word='It', pos='PP', lemma='it'),
 Tag(word='has', pos='VHZ', lemma='have'),
 Tag(word='been', pos='VBN', lemma='be'),
 Tag(word='a', pos='DT', lemma='a'),
 Tag(word='blessing', pos='NN', lemma='blessing'),
 Tag(word='for', pos='IN', lemma='for'),
 Tag(word='many', pos='JJ', lemma='many'),
 Tag(word='with', pos='IN', lemma='with'),
 Tag(word='no', pos='DT', lemma='no'),
 Tag(word='Medicaid', pos='NP', lemma='Medicaid'),
 Tag(word='expansion', pos='NN', lemma='expansion'),
 Tag(word='in', pos='IN', lemma='in'),
 Tag(word='Florida', pos='NP', lemma='Florida'),
 Tag(word='.', pos='SENT', lemma='.'),
 Tag(word='Most', pos='JJS', lemma='most'),
 Tag(word='jobs', pos='NNS', lemma='job'),
 Tag(word='do', pos='VVP', lemma='do'),
 Tag(word="n't", pos='RB', lemma="n't"),
 Tag(word='offer', pos='VV', lemma='offer'),
 Tag(word='insurance', pos='NN', lemma='insurance'),
 Tag(word='and', pos='CC', lemma='and'),
 Tag(word='many', pos='JJ', lemma='many'),
 Tag(word='onl', pos='NN', lemma='onl'),


In [69]:
twtTxt[14355]

'"Man, this @Bluejays team keeps getting better. David Price on the tit tomorrow!"'

In [70]:
### use subjective and objective tags to help the prediction

In [71]:
subjective = []
for i in range(len(twtTxt)):
    count = 0
    for tag in treetaggerwrapper.make_tags(tagger.tag_text(twtTxt[i])):
        try:
            if tag[1] == "PP" or tag[1] == "PP$" or tag[1] == "UH" or \
            tag[1] == "RB" or tag[1] == "RBR" or tag[1] == "RBS" or \
            tag[1] == "MD" or tag[1] == "VB":
                count += 1
        except IndexError:
            continue
    subjective.append(count)

In [72]:
len(subjective)

22987

In [73]:
objective = []
for i in range(len(twtTxt)):
    count = 0
    for tag in treetaggerwrapper.make_tags(tagger.tag_text(twtTxt[i])):
        try:
            if tag[1] == "NP" or tag[1] == "NPS" or tag[1] == "NNS" or \
            tag[1] == "WP$" or tag[1] == "JJR":
                count += 1
        except IndexError:
            continue
    objective.append(count)

In [74]:
len(objective)

22987

In [75]:
f1Df = pd.concat([twtDf, pd.DataFrame({"subjective":subjective, "objective":objective})], axis = 1)

In [76]:
f1Df  # FOR Training with new features

,a,amazing,antman,are,at,awesome,best,birthday,cant,cream,...,shit,so,stupid,supremacists,th,they,tomorrow,trump,objective,subjective
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2,6
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,3
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,2
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,5
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,0


In [77]:
devTwt = []
with open("./2017S1-KTproj2-data/dev-tweets.txt") as f:
    for line in f:
        devTwt.append(line.split("\t")[1])

In [78]:
subjectiveDev = []
for i in range(len(devTwt)):
    count = 0
    for tag in treetaggerwrapper.make_tags(tagger.tag_text(devTwt[i])):
        try:
            if tag[1] == "PP" or tag[1] == "PP$" or tag[1] == "UH" or \
            tag[1] == "RB" or tag[1] == "RBR" or tag[1] == "RBS" or \
            tag[1] == "MD" or tag[1] == "VB":
                count += 1
        except IndexError:
            continue
    subjectiveDev.append(count)

In [79]:
sum(subjectiveDev)

12166

In [80]:
objectiveDev = []
for i in range(len(devTwt)):
    count = 0
    for tag in treetaggerwrapper.make_tags(tagger.tag_text(devTwt[i])):
        try:
            if tag[1] == "NP" or tag[1] == "NPS" or tag[1] == "NNS" or \
            tag[1] == "WP$" or tag[1] == "JJR":
                count += 1
        except IndexError:
            continue
    objectiveDev.append(count)

In [81]:
sum(objectiveDev) # same type of distribution

21272

In [82]:
devEngDf = pd.concat([devForPred, pd.DataFrame({"subjective":subjectiveDev, "objective":objectiveDev})], axis = 1)

In [83]:
devEngDf.head()

,a,amazing,antman,are,at,awesome,best,birthday,cant,cream,...,shit,so,stupid,supremacists,th,they,tomorrow,trump,objective,subjective
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,4
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,4,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,3,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,3


In [84]:
clfEng = GaussianNB() # assume normal distribution
clfEng.fit(f1Df,sentiment)

GaussianNB(priors=None)

In [85]:
predNaiveDevelEng = clfEng.predict(devEngDf) 
NaiveEngCM = confusion_matrix(devSent, predNaiveDevelEng, labels = ["positive","negative","neutral"])
NaiveEngCM

array([[ 800,   41,  647],
       [ 159,  282,  597],
       [ 500,  172, 1728]])

In [86]:
print(classification_report(devSent,predNaiveDevelEng,labels = ["positive","negative","neutral"]))

             precision    recall  f1-score   support

   positive       0.55      0.54      0.54      1488
   negative       0.57      0.27      0.37      1038
    neutral       0.58      0.72      0.64      2400

avg / total       0.57      0.57      0.55      4926



In [87]:
accuracy_score(devSent,predNaiveDevelEng) # 2% improvements

0.57044254973609421

In [88]:
clfEngMult = MultinomialNB()
clfEngMult.fit(f1Df, sentiment)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [89]:
predNBEngDevelMult = clfEngMult.predict(devEngDf)
NBDevelMultCM = confusion_matrix(devSent,predNBEngDevelMult,labels = ["positive","negative","neutral"])
NBDevelMultCM

array([[ 613,   63,  812],
       [  97,  290,  651],
       [ 258,  213, 1929]])

In [90]:
print(classification_report(devSent,predNBEngDevelMult,labels = ["positive","negative","neutral"])) # slightly worse performance

             precision    recall  f1-score   support

   positive       0.63      0.41      0.50      1488
   negative       0.51      0.28      0.36      1038
    neutral       0.57      0.80      0.67      2400

avg / total       0.58      0.57      0.55      4926



In [91]:
accuracy_score(devSent,predNBEngDevelMult)

0.57490864799025576

In [92]:
svmClfEng = svm.LinearSVC()
predSvmEng = svmClfEng.fit(f1Df,sentiment).predict(devEngDf)

In [93]:
svmEngCM = confusion_matrix(devSent,predSvmEng,labels = ["positive","negative","neutral"])
svmEngCM

array([[ 534,   27,  927],
       [  65,  216,  757],
       [ 163,   99, 2138]])

In [94]:
print(classification_report(devSent,predSvmEng,labels = ["positive","negative","neutral"])) # slightly worse performance or no effect

             precision    recall  f1-score   support

   positive       0.70      0.36      0.47      1488
   negative       0.63      0.21      0.31      1038
    neutral       0.56      0.89      0.69      2400

avg / total       0.62      0.59      0.54      4926



In [95]:
accuracy_score(devSent,predSvmEng)

0.58627689809175798

In [96]:
######## use likelihood in tag and type of sentiment ######

In [97]:
negative = []
for i in range(len(twtTxt)):
    count = 0
    for tag in treetaggerwrapper.make_tags(tagger.tag_text(twtTxt[i])):
        try:
            if tag[1] == "WP$" or tag[1] == "POS" or tag[1] == "RBS" or tag[1] == "JJS":
                count += 1
        except IndexError:
            continue
    negative.append(count)

In [98]:
positive = []
for i in range(len(twtTxt)):
    count = 0
    for tag in treetaggerwrapper.make_tags(tagger.tag_text(twtTxt[i])):
        try:
            if tag[1] == "RB" or tag[1] == "WRB" or tag[1] == "VBD" or tag[1] == "VBN":
                count += 1
        except IndexError:
            continue
    positive.append(count)

In [99]:
sum(negative)

4143

In [100]:
sum(positive)

24362

In [101]:
negativeDev = []
for i in range(len(devTwt)):
    count = 0
    for tag in treetaggerwrapper.make_tags(tagger.tag_text(devTwt[i])):
        try:
            if tag[1] == "WP$" or tag[1] == "POS" or tag[1] == "RBS" or tag[1] == "JJS":
                count += 1
        except IndexError:
            continue
    negativeDev.append(count)

In [102]:
sum(negativeDev)

897

In [103]:
positiveDev = []
for i in range(len(devTwt)):
    count = 0
    for tag in treetaggerwrapper.make_tags(tagger.tag_text(devTwt[i])):
        try:
            if tag[1] == "RB" or tag[1] == "WRB" or tag[1] == "VBD" or tag[1] == "VBN":
                count += 1
        except IndexError:
            continue
    positiveDev.append(count)

In [104]:
sum(positiveDev)

5175

In [105]:
f2Df = featureEngDf = pd.concat([twtDf, pd.DataFrame({"positivity":positive, "negativity":negative})], axis = 1)
f2Df

,a,amazing,antman,are,at,awesome,best,birthday,cant,cream,...,shit,so,stupid,supremacists,th,they,tomorrow,trump,negativity,positivity
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
devEngDf2 = pd.concat([devForPred, pd.DataFrame({"positivity":positiveDev, "negativity":negativeDev})], axis = 1)
devEngDf2

,a,amazing,antman,are,at,awesome,best,birthday,cant,cream,...,shit,so,stupid,supremacists,th,they,tomorrow,trump,negativity,positivity
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
5,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
clfNBEng2 =  GaussianNB() # test 
predNBEng2 = clfNBEng2.fit(f2Df,sentiment).predict(devEngDf2)

In [108]:
print(classification_report(devSent,predNBEng2,labels = ["positive","negative","neutral"]))

             precision    recall  f1-score   support

   positive       0.54      0.56      0.55      1488
   negative       0.54      0.29      0.38      1038
    neutral       0.58      0.68      0.63      2400

avg / total       0.56      0.56      0.55      4926



In [109]:
accuracy_score(devSent,predNBEng2) # very slight improvements

0.56374340235485176

In [110]:
clfNBMultEng2 =  MultinomialNB() # test Multinomial
predNBMultEng2 = clfNBMultEng2.fit(f2Df,sentiment).predict(devEngDf2)

In [111]:
print(classification_report(devSent,predNBMultEng2,labels = ["positive","negative","neutral"]))

             precision    recall  f1-score   support

   positive       0.71      0.36      0.48      1488
   negative       0.61      0.22      0.32      1038
    neutral       0.56      0.88      0.68      2400

avg / total       0.61      0.58      0.54      4926



In [112]:
accuracy_score(devSent,predNBMultEng2)

0.58424685343077543

In [113]:
svmClfEng2 = svm.LinearSVC()
predSvmEng2 = svmClfEng2.fit(f2Df,sentiment).predict(devEngDf2)

In [114]:
print(classification_report(devSent,predSvmEng2,labels = ["positive","negative","neutral"]))

             precision    recall  f1-score   support

   positive       0.71      0.36      0.48      1488
   negative       0.63      0.22      0.32      1038
    neutral       0.56      0.89      0.69      2400

avg / total       0.62      0.59      0.55      4926



In [115]:
accuracy_score(devSent,predSvmEng2)

0.58952496954933009

In [116]:
f3Df = featureEngDf = pd.concat([twtDf, pd.DataFrame({
    "positivity":positive, "negativity":negative, "subjective":subjective, "objective":objective})], axis = 1)
f3Df

,a,amazing,antman,are,at,awesome,best,birthday,cant,cream,...,stupid,supremacists,th,they,tomorrow,trump,negativity,objective,positivity,subjective
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,3,2,2
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,3,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2,1,6
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,2,3
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,1,2
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,1,5
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6,0,0


In [117]:
devEngDf3 = pd.concat([devForPred, pd.DataFrame({
    "positivity":positiveDev, "negativity":negativeDev,
    "subjective":subjectiveDev, "objective":objectiveDev})], axis = 1)
devEngDf3

,a,amazing,antman,are,at,awesome,best,birthday,cant,cream,...,stupid,supremacists,th,they,tomorrow,trump,negativity,objective,positivity,subjective
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,2,4
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,4,1,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,3,1,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,3
5,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,4,1,3
6,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,2,2,2
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,3,4
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
9,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,2


In [118]:
clfNBEng3 =  GaussianNB() # test of Multinomial
predNBEng3 = clfNBEng3.fit(f3Df,sentiment).predict(devEngDf3)

In [119]:
print(classification_report(devSent,predNBEng3,labels = ["positive","negative","neutral"])) # same type of improvements

             precision    recall  f1-score   support

   positive       0.55      0.54      0.54      1488
   negative       0.56      0.27      0.36      1038
    neutral       0.58      0.72      0.64      2400

avg / total       0.57      0.57      0.55      4926



In [120]:
accuracy_score(devSent,predNBEng3)

0.56881851400730821

In [121]:
clfNBMultEng3 =  MultinomialNB() # test of Multinomial
predNBMultEng3 = clfNBMultEng3.fit(f3Df,sentiment).predict(devEngDf3)

In [122]:
print(classification_report(devSent,predNBMultEng3,labels = ["positive","negative","neutral"]))

             precision    recall  f1-score   support

   positive       0.64      0.41      0.50      1488
   negative       0.50      0.33      0.40      1038
    neutral       0.58      0.79      0.67      2400

avg / total       0.58      0.58      0.56      4926



In [123]:
accuracy_score(devSent,predNBMultEng3)

0.57714169711733654

In [124]:
svmClfEng3 = svm.LinearSVC()
predSvmEng3 = svmClfEng3.fit(f3Df,sentiment).predict(devEngDf3)

In [125]:
print(classification_report(devSent,predSvmEng3,labels = ["positive","negative","neutral"])) # literally no effect no SVM

             precision    recall  f1-score   support

   positive       0.71      0.36      0.48      1488
   negative       0.63      0.21      0.31      1038
    neutral       0.56      0.89      0.69      2400

avg / total       0.62      0.59      0.54      4926



In [126]:
accuracy_score(devSent,predSvmEng3) 

0.58668290702395454

In [127]:
################## C parameter tuning (i mean why not..) ###############

In [128]:
svmClftune = svm.LinearSVC(C = 5.0)
svmTune = svmClftune.fit(twtDf,sentiment).predict(devForPred)

In [129]:
print(classification_report(devSent,svmTune,labels = ["positive","negative","neutral"])) # essentially no effect

             precision    recall  f1-score   support

   positive       0.72      0.36      0.48      1488
   negative       0.64      0.21      0.31      1038
    neutral       0.56      0.90      0.69      2400

avg / total       0.63      0.59      0.55      4926



In [130]:
accuracy_score(devSent,svmTune)

0.59013398294762487

In [131]:
print(classification_report(devSent,predLSVC,labels = ["positive","negative","neutral"])) # from above

             precision    recall  f1-score   support

   positive       0.72      0.36      0.48      1488
   negative       0.64      0.21      0.31      1038
    neutral       0.56      0.90      0.69      2400

avg / total       0.62      0.59      0.55      4926



In [132]:
accuracy_score(devSent,predLSVC)

0.59013398294762487

In [133]:
devSent.value_counts()

neutral     2400
positive    1488
negative    1038
Name: sentiment, dtype: int64

In [134]:
2400/len(devSent)

0.48721071863581

In [135]:
weightSvm = svm.LinearSVC(class_weight = {"positive":0.25,"negative":0.25,"neutral":0.5})
predSvmEng4 = weightSvm.fit(f3Df,sentiment).predict(devEngDf3)

In [136]:
print(classification_report(devSent,predSvmEng4,labels = ["positive","negative","neutral"])) 

             precision    recall  f1-score   support

   positive       0.75      0.31      0.44      1488
   negative       0.68      0.16      0.26      1038
    neutral       0.55      0.93      0.69      2400

avg / total       0.64      0.58      0.52      4926



In [137]:
accuracy_score(devSent,predSvmEng4)

0.57775071051563132

In [138]:
predSvmWeight = weightSvm.fit(twtDf,sentiment).predict(devForPred)

In [139]:
print(classification_report(devSent,predSvmWeight,labels = ["positive","negative","neutral"])) # from above

             precision    recall  f1-score   support

   positive       0.76      0.31      0.44      1488
   negative       0.70      0.16      0.26      1038
    neutral       0.55      0.93      0.69      2400

avg / total       0.64      0.58      0.52      4926



In [140]:
accuracy_score(devSent,predSvmWeight)

0.58018676410881043

In [141]:
weightSVM = confusion_matrix(devSent,predSvmWeight,labels = ["positive","negative","neutral"])
weightSVM

array([[ 462,   11, 1015],
       [  36,  162,  840],
       [ 108,   58, 2234]])

In [142]:
balanceSvm = svm.LinearSVC(class_weight = "balanced")
predbalance = balanceSvm.fit(twtDf,sentiment).predict(devForPred)

In [143]:
balanceCM = confusion_matrix(devSent,predbalance,labels = ["positive","negative","neutral"])
balanceCM

array([[ 649,   83,  756],
       [  89,  350,  599],
       [ 261,  276, 1863]])

In [144]:
print(classification_report(devSent,predbalance,labels = ["positive","negative","neutral"])) # from above

             precision    recall  f1-score   support

   positive       0.65      0.44      0.52      1488
   negative       0.49      0.34      0.40      1038
    neutral       0.58      0.78      0.66      2400

avg / total       0.58      0.58      0.57      4926



In [145]:
testDf = pd.read_csv("./2017S1-KTproj2-data/test.csv")
testDf.drop('id', axis =1 ,inplace =True)
testDf.drop('sentiment', axis = 1 , inplace = True)
testDf

,a,amazing,antman,are,at,awesome,best,birthday,cant,cream,...,racist,see,shit,so,stupid,supremacists,th,they,tomorrow,trump
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [146]:
predTest = weightSvm.predict(testDf)

In [147]:
#with open('predTest.txt', 'w') as f:
#    read_data = f.write(str(list(predTest)))

In [148]:
# tweets for examples

In [154]:
twtTxt[7918]

'2009 #Topps AG Allen Ginter #JustinVerlander #82 #Tigers Cy Young https://t.co/XsG12pzfjv #Baseball #Pitcher https://t.co/Cr3nfPmlLA'

In [156]:
sentiment[7918]

'neutral'

In [164]:
twtTxt[9090]

"Live coverage of today's Supreme Court hearing on whether parliament or government can trigger article 50 https://t.co/baNmacHYBB"

In [165]:
list(twtDf.columns)

['a',
 'amazing',
 'antman',
 'are',
 'at',
 'awesome',
 'best',
 'birthday',
 'cant',
 'cream',
 'day',
 'death',
 'drone',
 'excited',
 'fake',
 'fuck',
 'fucking',
 'good',
 'great',
 'gucci',
 'happy',
 'hate',
 'i',
 'ice',
 'im',
 'is',
 'leftists',
 'liberals',
 'love',
 'my',
 'national',
 'nazi',
 'night',
 'not',
 'obama',
 'people',
 'racist',
 'see',
 'shit',
 'so',
 'stupid',
 'supremacists',
 'th',
 'they',
 'tomorrow',
 'trump']